# Notes

$E(W)= \frac{<\Psi_m|H|\Psi_m>}{<\Psi_m|\Psi_m>}$

want $ \nabla E(W^*)=0$

have $ \Psi_m(S;W)=exp[\sum_i a_i \sigma_i^z] \times \Pi_{i=1}^M F_i(S)$

where $F_i(S)=2\cosh(b_i+\sum_j W_{ij}\sigma_i^z) $

Also, $\alpha=\frac{M}{N}$ where M is the hidden layers and N is the actual visible layers.

Also note, $S=(S_1, S_2, ... S_N)$

So the weights are $W=[a_i,b_j,W_{ij}]$

#### From Stochastic methods:

$E_{loc}=\frac{<S|H|\Psi_m>}{\Psi_m(S)}$

$W(p+1)=W(p)-\lambda S^{-1}(p)F(p)$

$S=S_{k,k'}+\lambda(p)\delta_{k,k'}S_{k,k}$

$\lambda(p)=max(\lambda_0 b^p,\lambda_{min})$

with $\lambda_0=100$, $b=0.9$, and $\lambda_{min}=10^{-4}$

$S_{k,k'}(p)=<O_k^* O_{k'}>-<O_k^*><O_{k'}>$

$F_k(p)=<E_{loc} O_k^*>-<E_{loc}><O_k^*>$

$O_k(S)=\frac{1}{\Psi_m(S)}\partial_{W_k} \Psi_m(S)$

#### The possible Hamiltonians:

transverse-field Ising (TFI): $H_{TFI}=-h\sum_i \sigma_i^x - \sum_{<i,j>}\sigma_i^z \sigma_j^z$

antiferromagnetic Heisenberg (AFH): $H_{AFH}=\sum_{<i,j>} \sigma_i^x\sigma_j^x + \sigma_i^y \sigma_j^y + \sigma_i^z \sigma_j^z$

In [4]:
# %load ./include/header.py
import numpy as np
import matplotlib.pyplot as plt
import sys
import math
sys.path.append('./include')
import ml4s
import json
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.style.use('./include/notebook.mplstyle')
np.set_printoptions(linewidth=120)
ml4s._set_css_style('./include/bootstrap.css')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [ ]:
def wavecalc(a,b,W,S,M,N):
    F=np.zeros((M))
    
    for i in range(M):
        x=0
        for j in range(N):
            x+=W[i,j]*S[j]
        x+=b[i]
        F[i]=2*np.cosh(x)
        x=0
        y=1
        for i in range(N):
            x+=a[i]*S[i]
            y=y*F[i]

    Ψ=np.exp(x)*y
    return Ψ

In [23]:
N=4
M=N*1
α=M/N
W=np.random.uniform(low=0,high=1,size=(M,N))
b=np.random.uniform(low=0,high=1,size=(M,1))
a=np.random.uniform(low=0,high=1,size=(N,2))
S=np.random.randint(low=0,high=2,size=(N,1))
σ=np.array([[1,0],[0,-1]])

In [24]:
O=np.zeros((N,1))
for i in range(N):
    hold=0
    for j in range(M):
        hold+=W[j,i]*S[i]
    holder=hold+b[i]
    O[i]=S[i,0]*np.tanh(holder)

In [ ]:
Eloc=np.zeros((N,1))

### Gradient Descent from Class

In [ ]:
def f(w,A):
    return (1/2)*w.T @ A @ w 

In [ ]:
from jax import jacfwd
df_dw = jacfwd(f,argnums=0)

In [ ]:
# hyperparameters
η = 0.5
γ = 0.9
num_iter = 10

w = np.array([4.0,4.0])

ax.plot(*w, marker='.', color='k', ms=15)  
v = np.zeros(2)

for i in range(num_iter):
    
    # keep a copy for plotting
    w_old = np.copy(w)
    
    # perform the CM update
    v = γ*v + η*df_dw(w, A)
    w -= v

# Solving the quantum many-body problem with NQS
## 1D transverse field Ising model chain


In [7]:
def complexval(N):
    a=(np.random.random(N)-0.5)*0.01
    b=(np.random.random(N)-0.5)*0.01

In [16]:
N=4
alpha=2
M=alpha*N
h=2

a=complexval(N)
b=complexval(N)
W=complexval((N,M))
sigma=np.random.randint(2,size=N)
sigma[sigma==0]=-1

sigma_i=list(range(N))

In [17]:
def effectangle(sigma):
    return b+np.inner(W.T,sigma)

In [18]:
def Psi_M(sigma,a,b,W):
    return np.exp(np.inner(a,sigma))*np.prod(2*np.cosh(effectangle(sigma)))

In [19]:
def E_loc(sigma):
    E=0

    #sigmaz 
    for i in sigma_i:
        if i==N-1: E-=sigma[i]*sigma[0]
        else: E-=sigma[i]*sigma[i+1]

    #sigmax
    Psi_M_s=Psi_M(sigma,a,b,W)
    for i in sigma:
        sigma[i] *= -1
        E-= -h*Psi_M(sigma,a,b,W)/Psi_M_s
        sigma[i]*=-1
    
    return E/N

In [ ]:
def step():
    site=np.random.choice(sigma_i,1)
    Psi_M_i=Psi_M(sigma,a,b,W)

    for i in site:
        sigma[i]*=-1
    
    Psi_M_f=Psi_M(sigma,a,b,W)
    accept=np.real((Psi_M_f*np.conj(Psi_M_f))/(Psi_M_i*np.conj(Psi_M_i))

    if accpet<np.random.uniform():
        for i in site: state[i]*=1
        return 1
    else: return 0